In [1]:
% matplotlib inline


# Hyperparameter tuning with Ray Tune

Hyperparameter tuning can make the difference between an average model and a highly
accurate one. Often simple things like choosing a different learning rate or changing
a network layer size can have a dramatic impact on your model performance.

Fortunately, there are tools that help with finding the best combination of parameters.
[Ray Tune](https://docs.ray.io/en/latest/tune.html) is an industry standard tool for
distributed hyperparameter tuning. Ray Tune includes the latest hyperparameter search
algorithms, integrates with TensorBoard and other analysis libraries, and natively
supports distributed training through [Ray's distributed machine learning engine](https://ray.io/).

In this tutorial, we will show you how to integrate Ray Tune into your PyTorch
training workflow. We will extend [this tutorial from the PyTorch documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html) for training
a CIFAR10 image classifier.

As you will see, we only need to add some slight modifications. In particular, we
need to

1. wrap data loading and training in functions,
2. make some network parameters configurable,
3. add checkpointing (optional),
4. and define the search space for the model tuning

|

To run this tutorial, please make sure the following packages are
installed:

-  ``ray[tune]``: Distributed hyperparameter tuning library
-  ``torchvision``: For the data transformers

## Setup / Imports
Let's start with the imports:


In [2]:
from functools import partial
import numpy as np
import os

import torch
import torch.cuda
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import random_split
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms

from ray import tune
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler

from DataLoad.DataLoad import SetSplit, LoadNirtest
from Preprocessing.Preprocessing import Preprocessing
from WaveSelect.WaveSelcet import SpctrumFeatureSelcet
from Regression.Rgs import QuantitativeAnalysis
import multiprocessing
import programParameter.stringConfig as STRING_CONFIG
import programParameter.threadConfig as THREAD_CONFIG
from Regression.CNN import ZspPocessnew
from Evaluate.RgsEvaluate import ModelRgsevaluatePro

Most of the imports are needed for building the PyTorch model. Only the last three
imports are for Ray Tune.

## Data loaders
We wrap the data loaders in their own function and pass a global data directory.
This way we can share a data directory between different trials.



In [3]:
# def load_data(data_dir="./data"):
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
#     ])
#
#     trainset = torchvision.datasets.CIFAR10(
#         root=data_dir, train=True, download=True, transform=transform)
#
#     testset = torchvision.datasets.CIFAR10(
#         root=data_dir, train=False, download=True, transform=transform)
#
#     return trainset, testset

# todo: moved
def load_data(data, label, ProcessMethods, FslecetedMethods, SetSplitMethods):
    """
    这个函数主要服务于Ray Tune流水线
    :param data:
    :param label:
    :param ProcessMethods:
    :param FslecetedMethods:
    :param SetSplitMethods:
    :return:
    """
    processedData = Preprocessing(ProcessMethods, data)
    featureData, labels = SpctrumFeatureSelcet(FslecetedMethods, processedData, label)
    X_train, X_test, y_train, y_test = SetSplit(SetSplitMethods, featureData, labels, test_size=0.2, randomseed=123)
    return X_train, X_test, y_train, y_test

## Configurable neural network
We can only tune those parameters that are configurable. In this example, we can specify
the layer sizes of the fully connected layers:



In [4]:
# class Net(nn.Module):
#     def __init__(self, l1=120, l2=84):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(16 * 5 * 5, l1)
#         self.fc2 = nn.Linear(l1, l2)
#         self.fc3 = nn.Linear(l2, 10)
#
#     def forward(self, x):
#         x = self.pool(F.relu(self.conv1(x)))
#         x = self.pool(F.relu(self.conv2(x)))
#         x = x.view(-1, 16 * 5 * 5)
#         x = F.relu(self.fc1(x))
#         x = F.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x


# todo: moved
class Net(nn.Module):
    def __init__(self, acti_func, cnn_depth):
        super(Net, self).__init__()
        self.layers = nn.ModuleList([])
        input_channel = 1
        output_channel = 16
        for i in range(1, cnn_depth):
            self.layers.append(nn.Conv1d(input_channel, output_channel, 3, padding=1))
            self.layers.append(nn.BatchNorm1d(num_features=output_channel))
            self.layers.append(ac_dict[acti_func](inplace=True))
            self.layers.append(nn.MaxPool1d(2, 2))
            input_channel = output_channel
            output_channel = output_channel * 2
        # linear[c_num-1]
        self.reg = nn.Sequential(
            nn.Linear(649, 1000),  # 根据自己数据集修改
            nn.ReLU(inplace=True),
            nn.Linear(1000, 500),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(500, 1), )

    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        out = out.flatten(start_dim=1)
        # out = out.view(-1,self.output_channel)
        out = self.reg(out)
        return out

## The train function
Now it gets interesting, because we introduce some changes to the example [from the PyTorch
documentation](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html).

We wrap the training script in a function ``train_cifar(config, checkpoint_dir=None, data_dir=None)``.
As you can guess, the ``config`` parameter will receive the hyperparameters we would like to
train with. The ``checkpoint_dir`` parameter is used to restore checkpoints. The ``data_dir`` specifies
the directory where we load and store the data, so multiple runs can share the same data source.

```python
net = Net(config["l1"], config["l2"])

if checkpoint_dir:
    model_state, optimizer_state = torch.load(
        os.path.join(checkpoint_dir, "checkpoint"))
    net.load_state_dict(model_state)
    optimizer.load_state_dict(optimizer_state)
```
The learning rate of the optimizer is made configurable, too:

```python
optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
```
We also split the training data into a training and validation subset. We thus train on
80% of the data and calculate the validation loss on the remaining 20%. The batch sizes
with which we iterate through the training and test sets are configurable as well.

### Adding (multi) GPU support with DataParallel
Image classification benefits largely from GPUs. Luckily, we can continue to use
PyTorch's abstractions in Ray Tune. Thus, we can wrap our model in ``nn.DataParallel``
to support data parallel training on multiple GPUs:

```python
device = "cpu"
if torch.cuda.is_available():
    device = "cuda:0"
    if torch.cuda.device_count() > 1:
        net = nn.DataParallel(net)
net.to(device)
```
By using a ``device`` variable we make sure that training also works when we have
no GPUs available. PyTorch requires us to send our data to the GPU memory explicitly,
like this:

```python
for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
```
The code now supports training on CPUs, on a single GPU, and on multiple GPUs. Notably, Ray
also supports [fractional GPUs](https://docs.ray.io/en/master/using-ray-with-gpus.html#fractional-gpus)
so we can share GPUs among trials, as long as the model still fits on the GPU memory. We'll come back
to that later.

### Communicating with Ray Tune

The most interesting part is the communication with Ray Tune:

```python
with tune.checkpoint_dir(epoch) as checkpoint_dir:
    path = os.path.join(checkpoint_dir, "checkpoint")
    torch.save((net.state_dict(), optimizer.state_dict()), path)

tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
```
Here we first save a checkpoint and then report some metrics back to Ray Tune. Specifically,
we send the validation loss and accuracy back to Ray Tune. Ray Tune can then use these metrics
to decide which hyperparameter configuration lead to the best results. These metrics
can also be used to stop bad performing trials early in order to avoid wasting
resources on those trials.

The checkpoint saving is optional, however, it is necessary if we wanted to use advanced
schedulers like
[Population Based Training](https://docs.ray.io/en/master/tune/tutorials/tune-advanced-tutorial.html).
Also, by saving the checkpoint we can later load the trained models and validate them
on a test set.

### Full training function

The full code example looks like this:



In [5]:
# def train_cifar(config, checkpoint_dir=None, data_dir=None):
#     # 初始化模型
#     net = Net(config["l1"], config["l2"])
#
#     # 设置运行设备
#     device = "cpu"
#     if torch.cuda.is_available():
#         device = "cuda:0"
#         if torch.cuda.device_count() > 1:
#             net = nn.DataParallel(net)
#     net.to(device)
#
#     # 设置 损失函数
#     criterion = nn.CrossEntropyLoss()
#
#     # 设置 模型优化器
#     optimizer = optim.SGD(net.parameters(), lr=config["lr"], momentum=0.9)
#
#     # 多次训练之间共享数据
#     if checkpoint_dir:
#         model_state, optimizer_state = torch.load(
#             os.path.join(checkpoint_dir, "checkpoint"))
#         net.load_state_dict(model_state)
#         optimizer.load_state_dict(optimizer_state)
#
#     # 获得训练和测试数据集合
#     trainset, testset = load_data(data_dir)
#
#     test_abs = int(len(trainset) * 0.8)
#     # 数据集划分
#     train_subset, val_subset = random_split(
#         trainset, [test_abs, len(trainset) - test_abs])
#
#     trainloader = torch.utils.data.DataLoader(
#         train_subset,
#         batch_size=int(config["batch_size"]),
#         shuffle=True,
#         num_workers=8)
#     valloader = torch.utils.data.DataLoader(
#         val_subset,
#         batch_size=int(config["batch_size"]),
#         shuffle=True,
#         num_workers=8)
#
#     # 开始训练
#     for epoch in range(10):  # loop over the dataset multiple times
#         running_loss = 0.0
#         epoch_steps = 0
#         for i, data in enumerate(trainloader, 0):
#             # get the inputs; data is a list of [inputs, labels]
#             inputs, labels = data
#             inputs, labels = inputs.to(device), labels.to(device)
#
#             # zero the parameter gradients
#             optimizer.zero_grad()
#
#             # forward + backward + optimize
#             outputs = net(inputs)
#             loss = criterion(outputs, labels)
#             loss.backward()
#             optimizer.step()
#
#             # print statistics
#             running_loss += loss.item()
#             epoch_steps += 1
#             if i % 2000 == 1999:  # print every 2000 mini-batches
#                 print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
#                                                 running_loss / epoch_steps))
#                 running_loss = 0.0
#
#         # Validation loss
#         val_loss = 0.0
#         val_steps = 0
#         total = 0
#         correct = 0
#         for i, data in enumerate(valloader, 0):
#             with torch.no_grad():
#                 inputs, labels = data
#                 inputs, labels = inputs.to(device), labels.to(device)
#
#                 outputs = net(inputs)
#                 _, predicted = torch.max(outputs.data, 1)
#                 total += labels.size(0)
#                 correct += (predicted == labels).sum().item()
#
#                 loss = criterion(outputs, labels)
#                 val_loss += loss.cpu().numpy()
#                 val_steps += 1
#
#         with tune.checkpoint_dir(epoch) as checkpoint_dir:
#             path = os.path.join(checkpoint_dir, "checkpoint")
#             torch.save((net.state_dict(), optimizer.state_dict()), path)
#
#         tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
#     print("Finished Training")

# todo: moved
def train(config, checkpoint_dir=None, data_dir=None, loss_func=nn.MSELoss(), data_wrapUp=None):
    BATCH_SIZE = 16
    TBATCH_SIZE = 240

    # 实例化模型 （自动超参：激活函数、网络深度）
    network = Net(config[STRING_CONFIG.acti_func], config[STRING_CONFIG.cnn_depth])

    # 设置运行设备
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if torch.cuda.device_count() > 1:
            network = nn.DataParallel(network)
    network.to(device)

    # 设置 损失函数
    criterion = loss_func

    # 使用字典映射调用优化器
    OPTIM_DICT = {
        STRING_CONFIG.Adam: torch.optim.Adam(network.parameters(), lr=config[STRING_CONFIG.lr], weight_decay=0.001),
        STRING_CONFIG.SGD: torch.optim.SGD(network.parameters(), lr=config[STRING_CONFIG.lr],
                                           momentum=config[STRING_CONFIG.momentum]),
        # 接下来的激活函数 torch并不支持。
        # 'Adagrad': torch.optim.Adagrad(network.parameters(), lr=0.01),
        STRING_CONFIG.Adadelta: torch.optim.Adadelta(network.parameters()),
        STRING_CONFIG.RMSprop: torch.optim.RMSprop(network.parameters(), lr=config[STRING_CONFIG.lr], alpha=0.99),
        STRING_CONFIG.Adamax: torch.optim.Adamax(network.parameters(), lr=config[STRING_CONFIG.lr], betas=(0.9, 0.999)),
        # 'LBFGS': torch.optim.LBFGS(network.parameters(), lr=0.01)
    }
    # 设置 模型优化器 (自动超参：模型优化器)
    optimizer = OPTIM_DICT[config[STRING_CONFIG.optimizer]]
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer=optimizer,
                                                           mode="min",
                                                           factor=0.5, verbose=1,
                                                           eps=1e-06,
                                                           patience=20)

    # 多次训练之间共享数据
    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    # 获得（划分处理完成后的）训练和测试数据集合
    X_train, X_test, y_train, y_test = data_wrapUp

    # 数据集预处理以及数据集载入
    trainSet, testSet, yscaler = ZspPocessnew(X_train, X_test, y_train, y_test, need=True)

    test_abs = int(len(trainSet) * 0.8)
    # 数据集划分
    train_subset, val_subset = random_split(trainSet, [test_abs, len(trainSet) - test_abs])
    train_loader = torch.utils.data.DataLoader(dataset=train_subset,
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               num_workers=THREAD_CONFIG.num_workers)
    val_loader = torch.utils.data.DataLoader(dataset=val_subset,
                                             batch_size=TBATCH_SIZE,
                                             shuffle=True,
                                             num_workers=THREAD_CONFIG.num_workers)

    print("Start Training!\n")
    # 以 per epoch 为单位记录模型训练损失 & 以 per iteration 为单位记录模型训练损失
    trainIter_losses = []
    epoch_loss = []
    for epoch in range(config[STRING_CONFIG.epochs]):
        train_rmse = []
        train_r2 = []
        train_mae = []
        avg_loss = []

        ################### 记录以epoch来记录 loss ###################
        temp_trainLosses = 0
        ################### 记录以epoch来记录 loss ###################

        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(train_loader, 0):
            # 获得输入，data is a list of [inputs, labels]
            inputs, labels = data  # 输入和标签都等于data
            inputs = Variable(inputs).type(torch.FloatTensor).to(device)  # batch x
            labels = Variable(labels).type(torch.FloatTensor).to(device)  # batch y

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = network(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            pred = outputs.detach().cpu().numpy()
            y_true = labels.detach().cpu().numpy()
            trainIter_losses.append(loss.item())
            temp_trainLosses = loss.item()
            rmse, R2, mae = ModelRgsevaluatePro(pred, y_true, yscaler)
            avg_train_loss = np.mean(trainIter_losses)
            train_rmse.append(rmse)
            train_r2.append(R2)
            train_mae.append(mae)

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        epoch_loss.append(temp_trainLosses)
        avgrmse = np.mean(train_rmse)
        avgr2 = np.mean(train_r2)
        avgmae = np.mean(train_mae)
        # print('Epoch:{}, TRAIN:rmse:{}, R2:{}, mae:{}'.format((epoch + 1), (avgrmse), (avgr2), (avgmae)))
        # print('lr:{}, avg_train_loss:{}'.format((optimizer.param_groups[0]['lr']), avg_train_loss))

        avg_loss.append(np.array(avg_train_loss))

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0

        test_rmse = []
        test_r2 = []
        test_mae = []
        for i, data in enumerate(val_loader, 0):
            with torch.no_grad():
                inputs, labels = data  # 输入和标签都等于data
                inputs = Variable(inputs).type(torch.FloatTensor).to(device)  # batch x
                labels = Variable(labels).type(torch.FloatTensor).to(device)  # batch y

                outputs = network(inputs)
                pred = outputs.detach().cpu().numpy()
                # y_pred.append(pred.astype(int))
                y_true = labels.detach().cpu().numpy()
                # y.append(y_true.astype(int))
                rmse, R2, mae = ModelRgsevaluatePro(pred, y_true, yscaler)
                test_rmse.append(rmse)
                test_r2.append(R2)
                test_mae.append(mae)

                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

            avgrmse = np.mean(test_rmse)
            avgr2 = np.mean(test_r2)
            avgmae = np.mean(test_mae)
            # print('EPOCH：{}, TEST: rmse:{}, R2:{}, mae:{}\n'.format((epoch + 1), (avgrmse), (avgr2), (avgmae)))
            # 将每次测试结果实时写入acc.txt文件中
            scheduler.step(rmse)

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((network.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

As you can see, most of the code is adapted directly from the original example.

## Test set accuracy
Commonly the performance of a machine learning model is tested on a hold-out test
set with data that has not been used for training the model. We also wrap this in a
function:



In [6]:
# def test_accuracy(net, device="cpu"):
#     trainset, testset = load_data()
#
#     testloader = torch.utils.data.DataLoader(
#         testset, batch_size=4, shuffle=False, num_workers=2)
#
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for data in testloader:
#             images, labels = data
#             images, labels = images.to(device), labels.to(device)
#             outputs = net(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()
#
#     return correct / total

# todo: moved
def test_accuracy(net, device="cpu", data_wrapUp=None):
    X_train, X_test, y_train, y_test = data_wrapUp

    # 数据集预处理以及数据集载入
    trainSet, testSet, yscaler = ZspPocessnew(X_train, X_test, y_train, y_test, need=True)

    testloader = torch.utils.data.DataLoader(
        testset, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total


The function also expects a ``device`` parameter, so we can do the
test set validation on a GPU.

## Configuring the search space
Lastly, we need to define Ray Tune's search space. Here is an example:

```python
config = {
    "l1": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "l2": tune.sample_from(lambda _: 2**np.random.randint(2, 9)),
    "lr": tune.loguniform(1e-4, 1e-1),
    "batch_size": tune.choice([2, 4, 8, 16])
}
```
The ``tune.sample_from()`` function makes it possible to define your own sample
methods to obtain hyperparameters. In this example, the ``l1`` and ``l2`` parameters
should be powers of 2 between 4 and 256, so either 4, 8, 16, 32, 64, 128, or 256.
The ``lr`` (learning rate) should be uniformly sampled between 0.0001 and 0.1. Lastly,
the batch size is a choice between 2, 4, 8, and 16.

At each trial, Ray Tune will now randomly sample a combination of parameters from these
search spaces. It will then train a number of models in parallel and find the best
performing one among these. We also use the ``ASHAScheduler`` which will terminate bad
performing trials early.

We wrap the ``train_cifar`` function with ``functools.partial`` to set the constant
``data_dir`` parameter. We can also tell Ray Tune what resources should be
available for each trial:

```python
gpus_per_trial = 2
# ...
result = tune.run(
    partial(train_cifar, data_dir=data_dir),
    resources_per_trial={"cpu": 8, "gpu": gpus_per_trial},
    config=config,
    num_samples=num_samples,
    scheduler=scheduler,
    progress_reporter=reporter,
    checkpoint_at_end=True)
```
You can specify the number of CPUs, which are then available e.g.
to increase the ``num_workers`` of the PyTorch ``DataLoader`` instances. The selected
number of GPUs are made visible to PyTorch in each trial. Trials do not have access to
GPUs that haven't been requested for them - so you don't have to care about two trials
using the same set of resources.

Here we can also specify fractional GPUs, so something like ``gpus_per_trial=0.5`` is
completely valid. The trials will then share GPUs among each other.
You just have to make sure that the models still fit in the GPU memory.

After training the models, we will find the best performing one and load the trained
network from the checkpoint file. We then obtain the test set accuracy and report
everything by printing.

The full main function looks like this:



In [ ]:
# def main(num_samples=10, max_num_epochs=10, gpus_per_trial=2):
#     data_dir = os.path.abspath("./data")
#     load_data(data_dir)
#     config = {
#         "l1": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
#         "l2": tune.sample_from(lambda _: 2 ** np.random.randint(2, 9)),
#         "lr": tune.loguniform(1e-4, 1e-1),
#         "batch_size": tune.choice([2, 4, 8, 16])
#     }
#     scheduler = ASHAScheduler(
#         metric="loss",
#         mode="min",
#         max_t=max_num_epochs,
#         grace_period=1,
#         reduction_factor=2)
#     reporter = CLIReporter(
#         # parameter_columns=["l1", "l2", "lr", "batch_size"],
#         metric_columns=["loss", "accuracy", "training_iteration"])
#     result = tune.run(
#         partial(train_cifar, data_dir=data_dir),
#         resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
#         config=config,
#         num_samples=num_samples,
#         scheduler=scheduler,
#         progress_reporter=reporter)
#
#     best_trial = result.get_best_trial("loss", "min", "last")
#     print("Best trial config: {}".format(best_trial.config))
#     print("Best trial final validation loss: {}".format(
#         best_trial.last_result["loss"]))
#     print("Best trial final validation accuracy: {}".format(
#         best_trial.last_result["accuracy"]))
#
#     best_trained_model = Net(best_trial.config["l1"], best_trial.config["l2"])
#     device = "cpu"
#     if torch.cuda.is_available():
#         device = "cuda:0"
#         if gpus_per_trial > 1:
#             best_trained_model = nn.DataParallel(best_trained_model)
#     best_trained_model.to(device)
#
#     best_checkpoint_dir = best_trial.checkpoint.value
#     model_state, optimizer_state = torch.load(os.path.join(
#         best_checkpoint_dir, "checkpoint"))
#     best_trained_model.load_state_dict(model_state)
#
#     test_acc = test_accuracy(best_trained_model, device)
#     print("Best trial test set accuracy: {}".format(test_acc))
#
#
# if __name__ == "__main__":
#     # You can change the number of GPUs per trial here:
#     main(num_samples=10, max_num_epochs=10, gpus_per_trial=0)

# todo: moved
def autoTuneMain(num_samples=10, max_num_epochs=10, gpus_per_trial=1, data=None, label=None,
                 ProcessMethods=None, FslecetedMethods=None, SetSplitMethods=None, model=None,
                 cnn_depth=None, loss=None, config=None, max_cpu_cores=1):
    # load data
    data_wrapUp = load_data(data=data, label=label,
                            ProcessMethods=ProcessMethods,
                            FslecetedMethods=FslecetedMethods,
                            SetSplitMethods=SetSplitMethods)

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)

    reporter = CLIReporter(
        parameter_columns=[STRING_CONFIG.cnn_depth, STRING_CONFIG.epochs, STRING_CONFIG.lr, STRING_CONFIG.momentum,
                           STRING_CONFIG.optimizer, STRING_CONFIG.acti_func],
        metric_columns=["loss", "accuracy", "training_iteration"])

    # 使用字典映射调用 loss函数
    LOSS_DICT = {
        STRING_CONFIG.MSE: nn.MSELoss(),
        STRING_CONFIG.L1: nn.L1Loss(),
        STRING_CONFIG.CrossEntropy: nn.CrossEntropyLoss(ignore_index=-100),
        STRING_CONFIG.Poisson: nn.PoissonNLLLoss(log_input=True, full=False, eps=1e-08),
        # 'KLDiv': nn.KLDivLoss(reduction='batchmean'),
    }

    result = tune.run(
        partial(train, data_wrapUp=data_wrapUp),
        resources_per_trial={"cpu": max_cpu_cores, "gpu": gpus_per_trial},
        config=config,
        num_samples=num_samples,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    # ============================= 自动超参完成后 ====================================
    best_trained_model = Net(best_trial.config[STRING_CONFIG.acti_func], best_trial.config[STRING_CONFIG.cnn_depth])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda:0"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)

    test_acc = test_accuracy(best_trained_model, device, data_wrapUp=data_wrapUp)
    print("Best trial test set accuracy: {}".format(test_acc))
    # ============================= 自动超参完成后 ====================================


# todo: moved
if __name__ == '__main__':
    # 载入原始数据并可视化
    data2, label2 = LoadNirtest('Rgs')

    # 是否使用自动超参功能
    is_autoTune = True
    if is_autoTune:
        # 获得本设备CPU最大核心数量。
        #   注意：如果出现可用内存空间不足，请自行调整`max_cpu_cores`以减少CPU核心数的使用！
        max_cpu_cores = multiprocessing.cpu_count()
        print("CPU has " + str(max_cpu_cores) + " cores on this device.")

        # 设置环境变量，解决Ray Tune因为CPU检测不准确而给出的warning。
        os.environ['RAY_USE_MULTIPROCESSING_CPU_COUNT'] = '1'
        # 设置超参数搜索空间
        search_space = {
            STRING_CONFIG.cnn_depth: tune.grid_search([1, 2, 3, 4, 5, 6, 7, 8]),
            STRING_CONFIG.epochs: tune.grid_search([10, 20, 30]),  # 具体EPOCHS搜参空间请根据模型的收敛情况判断而定
            STRING_CONFIG.lr: tune.grid_search([.001, .01, .1]),
            STRING_CONFIG.momentum: tune.grid_search([.5, .9, .99]),
            STRING_CONFIG.optimizer: tune.grid_search(["Adam", "SGD", "Adadelta", "RMSprop", "Adamax"]),
            STRING_CONFIG.acti_func: tune.choice(["relu", "lrelu"])
        }
        autoTuneMain(num_samples=10, max_num_epochs=10, gpus_per_trial=1,
                     data=data2, label=label2, ProcessMethods="MMS", FslecetedMethods="None",
                     SetSplitMethods="random", model="CNN_vgg", cnn_depth=5,
                     loss=STRING_CONFIG.MSE, config=search_space, max_cpu_cores=max_cpu_cores)
    else:
        # 光谱定量分析演示
        # 示意1: 预处理算法:MSC , 波长筛选算法: Uve, 数据集划分:KS, 定性分量模型: SVR
        # 这里我改了参数，10,'relu',5,'MSE','Adam'，10 是epoch，relu是激活函数，可以选。5是CNN有5层，MSE是损失函数，Adam是优化，这些可以到CNN.p文件看一下有什么选择。
        SpectralQuantitativeAnalysis(data=data2, label=label2,
                                     ProcessMethods="MMS", FslecetedMethods="None",
                                     SetSplitMethods="random",
                                     model="CNN_vgg", EPOCH=3, acti='relu',
                                     cnn_depth=5, loss='MSE', optim='SGD',
                                     is_autoTune=False,
                                     autoHyperConfig=search_space)
        # RMSE, R2, MAE = SpectralQuantitativeAnalysis(data=data2, label=label2, ProcessMethods="MMS",
        #                                              FslecetedMethods="None", SetSplitMethods="random",
        #                                              model="CNN_vgg", EPOCH=3, acti='relu', c_num=5,
        #                                              loss='MSE', optim='Adam')

        # print("The RMSE:{} R2:{}, MAE:{} of result!".format(RMSE, R2, MAE))


(655, 649)
(655,)
CPU has 56 cores on this device.


2023-02-06 02:54:39,908	INFO utils.py:616 -- Detected RAY_USE_MULTIPROCESSING_CPU_COUNT=1: Using multiprocessing.cpu_count() to detect the number of CPUs. This may be inconsistent when used inside docker. To correctly detect CPUs, unset the env var: `RAY_USE_MULTIPROCESSING_CPU_COUNT`.
2023-02-06 02:54:40,091	INFO worker.py:1529 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 
/root/miniconda3/envs/tf/lib/python3.8/site-packages/ray/tune/trainable/function_trainable.py:610: DeprecationWarning: `checkpoint_dir` in `func(config, checkpoint_dir)` is being deprecated. To save and load checkpoint in trainable functions, please use the `ray.air.session` API:

from ray.air import session

def train(config):
    # ...
    session.report({"metric": metric}, checkpoint=checkpoint)

For more information please see https://docs.ray.io/en/master/tune/api_docs/trainable.html

  warnings.warn(


== Status ==
Current time: 2023-02-06 02:54:52 (running for 00:00:02.62)
Memory usage on this node: 30.4/125.8 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 56.0/56 CPUs, 1.0/1 GPUs, 0.0/7.86 GiB heap, 0.0/3.93 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/train_2023-02-06_02-54-49
Number of trials: 61/10800 (60 PENDING, 1 RUNNING)
+-------------------+----------+-------------------+-------------+----------+-------+------------+-------------+-----------------------+
| Trial name        | status   | loc               |   cnn_depth |   EPOCHS |    lr |   momentum | optimizer   | activation_function   |
|-------------------+----------+-------------------+-------------+----------+-------+------------+-------------+-----------------------|
| train_9102e_00000 | RUNNING  | 172.17.0.7:247174 |           1 |       10 | 0.001 |        0.5 | Adam        | lrelu          

Trial name,accuracy,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,loss,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,training_iteration,trial_id,warmup_time
train_9102e_00000,1,2023-02-06_02-55-24,True,,3a5a97f149e341eaaa209a3cd34cc50e,autodl-container-7e4011b6ac-94b7816d,10,0.0222595,172.17.0.7,247174,True,27.3808,2.22793,27.3808,1675623324,0,,10,9102e_00000,0.0062387


== Status ==
Current time: 2023-02-06 02:55:08 (running for 00:00:19.07)
Memory usage on this node: 34.5/125.8 GiB 
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: -0.07869820296764374 | Iter 1.000: -0.15137840807437897
Resources requested: 56.0/56 CPUs, 1.0/1 GPUs, 0.0/7.86 GiB heap, 0.0/3.93 GiB objects (0.0/1.0 accelerator_type:G)
Result logdir: /root/ray_results/train_2023-02-06_02-54-49
Number of trials: 62/10800 (61 PENDING, 1 RUNNING)
+-------------------+----------+-------------------+-------------+----------+-------+------------+-------------+-----------------------+---------+------------+----------------------+
| Trial name        | status   | loc               |   cnn_depth |   EPOCHS |    lr |   momentum | optimizer   | activation_function   |    loss |   accuracy |   training_iteration |
|-------------------+----------+-------------------+-------------+----------+-------+------------+-------------+-----------------------+-----

If you run the code, an example output could look like this:

::

    Number of trials: 10 (10 TERMINATED)
    +-----+------+------+-------------+--------------+---------+------------+--------------------+
    | ... |   l1 |   l2 |          lr |   batch_size |    loss |   accuracy | training_iteration |
    |-----+------+------+-------------+--------------+---------+------------+--------------------|
    | ... |   64 |    4 | 0.00011629  |            2 | 1.87273 |     0.244  |                  2 |
    | ... |   32 |   64 | 0.000339763 |            8 | 1.23603 |     0.567  |                  8 |
    | ... |    8 |   16 | 0.00276249  |           16 | 1.1815  |     0.5836 |                 10 |
    | ... |    4 |   64 | 0.000648721 |            4 | 1.31131 |     0.5224 |                  8 |
    | ... |   32 |   16 | 0.000340753 |            8 | 1.26454 |     0.5444 |                  8 |
    | ... |    8 |    4 | 0.000699775 |            8 | 1.99594 |     0.1983 |                  2 |
    | ... |  256 |    8 | 0.0839654   |           16 | 2.3119  |     0.0993 |                  1 |
    | ... |   16 |  128 | 0.0758154   |           16 | 2.33575 |     0.1327 |                  1 |
    | ... |   16 |    8 | 0.0763312   |           16 | 2.31129 |     0.1042 |                  4 |
    | ... |  128 |   16 | 0.000124903 |            4 | 2.26917 |     0.1945 |                  1 |
    +-----+------+------+-------------+--------------+---------+------------+--------------------+


    Best trial config: {'l1': 8, 'l2': 16, 'lr': 0.00276249, 'batch_size': 16, 'data_dir': '...'}
    Best trial final validation loss: 1.181501
    Best trial final validation accuracy: 0.5836
    Best trial test set accuracy: 0.5806

Most trials have been stopped early in order to avoid wasting resources.
The best performing trial achieved a validation accuracy of about 58%, which could
be confirmed on the test set.

So that's it! You can now tune the parameters of your PyTorch models.

